## Final Results

In [ ]:
import pandas as pd
import numpy as np
import json

data = json.load(open("experiment_results.json", "r"))

# Flattening the nested dictionary
rows = []
for arch, mus in data.items():
    for mu, datasets in mus.items():
        for dataset, soft_targets_dict in datasets.items():
            for soft_target_key, metrics in hard_targets_dict.items():
                # Extracting 'True'/'False' from the key 'softtargets_True'
                soft_target_val = soft_target_key.replace("softtargets_", "")
                
                rows.append({
                    "Architecture": arch,
                    "MU": mu,
                    "Data": dataset,
                    "SoftTargets": soft_target_val,
                    "dAccuracy": metrics["accuracies"][0],
                    "ParamChange": metrics["param_changes"][0]
                })

df = pd.DataFrame(rows)
df

,Architecture,MU,Data,SoftTargets,dAccuracy,ParamChange
0,cnn,nova,mnist,True,0.830824,NaN
1,cnn,nova,mnist,False,0.071107,1.101459e+02
2,cnn,nova,fashion_mnist,True,0.778200,NaN
3,cnn,nova,fashion_mnist,False,0.032500,9.525878e+00
4,cnn,graddiff,mnist,True,0.068770,4.643958e+01
5,cnn,graddiff,mnist,False,0.231426,7.499043e+01
6,cnn,graddiff,fashion_mnist,True,0.174500,2.401609e+01
7,cnn,graddiff,fashion_mnist,False,0.156800,2.829250e+01
8,cnn,gradasc,mnist,True,0.052345,1.346588e+01
9,cnn,gradasc,mnist,False,0.061540,1.363111e+01


### Soft vs Hard

In [26]:
df_soft = df[df["SoftTargets"] == "True"]
df_hard = df[df["SoftTargets"] == "False"]

df_soft_accs = df_soft.dAccuracy.mean()
df_soft_param_changes = df_soft.ParamChange.mean()

df_soft_accs_median = df_soft.dAccuracy.median()
df_soft_param_changes_median = df_soft.ParamChange.median()

# Variance
df_soft_accs_var = df_soft.dAccuracy.var()
df_soft_param_changes_var = df_soft.ParamChange.var()

print(f"Mean dAccuracy: {df_soft_accs:.4f} | Mean param change: {df_soft_param_changes:.4f}")
print(f"Median dAccuracy: {df_soft_accs_median:.4f} | Median param change: {df_soft_param_changes_median:.4f}")
print(f"Variance dAccuracy: {df_soft_accs_var:.4f} | Variance param change: {df_soft_param_changes_var:.4f}")

Mean dAccuracy: 0.3147 | Mean param change: 155114122.7764
Median dAccuracy: 0.0839 | Median param change: 18.7410
Variance dAccuracy: 0.1330 | Variance param change: 240387264926605152.0000


In [27]:
df_hard_accs = df_hard.dAccuracy.mean()
df_hard_param_changes = df_hard.ParamChange.mean()

df_hard_accs_median = df_soft.dAccuracy.median()
df_hard_param_changes_median = df_hard.ParamChange.median()

# Variance
df_hard_accs_var = df_hard.dAccuracy.var()
df_hard_param_changes_var = df_hard.ParamChange.var()

print(f"Mean dAccuracy: {df_hard_accs:.4f} | Mean param change: {df_hard_param_changes:.4f}")
print(f"Median dAccuracy: {df_hard_accs_median:.4f} | Median param change: {df_hard_param_changes_median:.4f}")
print(f"Variance dAccuracy: {df_hard_accs_var:.4f} | Variance param change: {df_hard_param_changes_var:.4f}")

Mean dAccuracy: 0.1242 | Mean param change: 49.9996
Median dAccuracy: 0.0839 | Median param change: 25.9466
Variance dAccuracy: 0.0224 | Variance param change: 2540.6945


### Which Algorithm is the outlier?

In [23]:
df_nova_soft = df_soft[df_soft["MU"] == "nova"]
df_nova_hard = df_hard[df_hard["MU"] == "nova"]

df_nova_soft

,Architecture,MU,Data,SoftTargets,dAccuracy,ParamChange
0,cnn,nova,mnist,True,0.830824,NaN
2,cnn,nova,fashion_mnist,True,0.778200,NaN
20,mlp,nova,mnist,True,0.884418,1.550512e+09
22,mlp,nova,fashion_mnist,True,0.719300,6.285965e+05


In [24]:
df_nova_hard

,Architecture,MU,Data,SoftTargets,dAccuracy,ParamChange
1,cnn,nova,mnist,False,0.071107,110.145915
3,cnn,nova,fashion_mnist,False,0.032500,9.525878
21,mlp,nova,mnist,False,0.562067,164.394927
23,mlp,nova,fashion_mnist,False,0.090000,80.403513


___


In [1]:
from clearml import Task

def load_model_from_task(task_id: str, artifact_name: str = "trained Model"):
    """
    Connects to a ClearML Task and retrieves a pickled model artifact (e.g. from a Pipeline step).
    
    Args:
        task_id (str): The ID of the specific task (e.g. the "Model Training" step).
        artifact_name (str): The name of the artifact to retrieve. Defaults to "trained Model" 
                             (matches 'return_values' in the pipeline component).

    Returns:
        The deserialized model object (e.g. LitResNet).
    """
    print(f"Connecting to task: {task_id}")
    task = Task.get_task(task_id=task_id)
    
    if artifact_name in task.artifacts:
        print(f"Found artifact: '{artifact_name}'. Downloading and deserializing...")
        # .get() downloads the pickle and returns the Python object
        return task.artifacts[artifact_name].get()
    else:
        available_artifacts = list(task.artifacts.keys())
        raise ValueError(
            f"Artifact '{artifact_name}' not found in task {task_id}.\n"
            f"Available artifacts: {available_artifacts}"
        )
    
baseline    = load_model_from_task("3cebe0e4059e4fa58c57bd9a650ef7f5", "Baseline Model")
unlearn_ds  = load_model_from_task("ce89c3a50dda4ef1809314d2bce71374", "Test Dataloader")
test_dl     = load_model_from_task("ce89c3a50dda4ef1809314d2bce71374", "Unlearning Dataset")

Connecting to task: 3cebe0e4059e4fa58c57bd9a650ef7f5
Found artifact: 'Baseline Model'. Downloading and deserializing...


Exception 'Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.' encountered when getting artifact with type pickle and content type application/pickle


Connecting to task: ce89c3a50dda4ef1809314d2bce71374
Found artifact: 'Test Dataloader'. Downloading and deserializing...
Connecting to task: ce89c3a50dda4ef1809314d2bce71374
Found artifact: 'Unlearning Dataset'. Downloading and deserializing...


In [ ]:
from clearml import Task, PipelineController
import torch

def load_artifacts_from_pipeline(pipeline_id: str):
    """
    Fetches the necessary artifacts (Model, Unlearning Dataset, Test Loader) 
    from a specific ClearML Pipeline execution for finetuning the unlearning step.
    
    Args:
        pipeline_id (str): The ID of the pipeline controller task (from the Web UI).
        
    Returns:
        tuple: (target_model, unlearn_ds, test_loader)
    """
    print(f"Connecting to Pipeline: {pipeline_id}...")
    pipeline_task = Task.get_task(task_id=pipeline_id)
    
    # Get the list of steps (tasks) created by this pipeline
    # The pipeline controller tracks which tasks it created.
    # We look for tasks named "Train Baseline" and "Preprocess Data" 
    # that belong to this pipeline instance.
    
    # 1. Find the Preprocessing Task (outputs datasets)
    preprocess_task = None
    # 2. Find the Training Task (outputs the model)
    train_task = None

    # Iterate through child tasks to find the correct steps
    # Note: ClearML pipelines usually name steps as "StepName.TaskName" or similar, 
    # but strictly we search by the task names defined in your PipelineDecorator.
    for step in pipeline_task.get_pipeline_details().get('steps', []):
        step_name = step['name']
        task_id = step['task_id']
        
        if step_name == "Preprocess Data":
            preprocess_task = Task.get_task(task_id=task_id)
        elif step_name == "Train Baseline":
            train_task = Task.get_task(task_id=task_id)

    if not preprocess_task or not train_task:
        raise ValueError("Could not find 'Preprocess Data' or 'Train Baseline' steps in this pipeline.")

    print(f"Found Preprocess Task: {preprocess_task.id}")
    print(f"Found Training Task: {train_task.id}")

    # --- Load Artifacts ---
    
    # 1. Load Datasets from "Preprocess Data"
    # The artifact names must match the 'return_values' in your @PipelineDecorator.component
    print("Loading 'Unlearning Dataset'...")
    unlearn_ds = preprocess_task.artifacts['Unlearning Dataset'].get()
    
    print("Loading 'Test Dataloader'...")
    test_loader = preprocess_task.artifacts['Test Dataloader'].get()

    # 2. Load Model from "Train Baseline"
    # artifact name matches 'return_values=["Baseline Model", ...]'
    print("Loading 'Baseline Model'...")
    target_model = train_task.artifacts['Baseline Model'].get()

    print("✅ All artifacts loaded successfully.")
    return target_model, unlearn_ds, test_loader

ID = "dd5a479f9048425481e993af74648693"
_, _ , _ = load_artifacts_from_pipeline(ID)

Connecting to Pipeline: dd5a479f9048425481e993af74648693...


AttributeError: 'Task' object has no attribute 'get_pipeline_details'

In [ ]:
import torch

tensor = torch.tensor([[1.0, 2.0, 3.0]])
tensor.shape

f = [8]
f.extend(tensor.squeeze(0).shape)
v = torch.randn(f) # Original vector of size 10
v.shape

torch.Size([8, 1, 3])

In [ ]:
from src.data.dataset_loaders import TrainTestDataset, UnlearningDataLoader, UnlearningPairDataset
from torch.utils.data import DataLoader

# Create the dataset
unlearning_train_set = UnlearningPairDataset(
    csv_file="data/mnist_index.csv", 
    root_dir="data/softtarget_dataset/mnist",
    split='test'
)

# Use the custom DataLoader
unlearning_loader = UnlearningDataLoader(
    unlearning_train_set, 
    batch_size=2, 
    shuffle=True
)

Loading dataset index from data/mnist_index.csv...
Loaded Unlearning Dataset (test split using 'f1_split'): 889 Forget samples and 9111 Non-Forget samples.


In [ ]:
len(unlearning_loader)

3151

In [ ]:
unlearning_loader.dataset.tensor_to_label

{tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]): '0',
 tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]): '1',
 tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]): '2',
 tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]): '3',
 tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]): '4',
 tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]): '5',
 tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]): '6',
 tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]): '7',
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]): '8',
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]): '9'}

In [ ]:
ttd = TrainTestDataset(
    csv_file="data/mnist_index.csv", 
    root_dir="data/softtarget_dataset/mnist",
    split='test',
    sample_mode='forget',
    classes=['7', '8', '9']
)



Loading dataset index from data/mnist_index.csv...
Loaded test split (mode='forget', classes=['7', '8', '9']) with 6302 samples.
